# Protected Area Boundary Change

This notebook outlines the methodology used to measure at protected area boundaries via GEE. 

The notebook queries MODIS imagery and returns the gradient values of relevant bands as well as the vegetation indices NDVI and EVI. This code expects an annual time span and returns a geoTIFF for each band in each park for that year.

In [1]:
import ee
from utils import GeometryOperations, ImageOperations, Visualization
from config import *

In [2]:
service_account = 'jupyter-gee-project@ee-avs-dse.iam.gserviceaccount.com'
key_path = '../service_account_key.json'

credentials = ee.ServiceAccountCredentials(service_account, key_path)
ee.Initialize(credentials)

print(ee.String('Hello from the Earth Engine servers!').getInfo())

Hello from the Earth Engine servers!


## Class Definitions

In [3]:
def main(protected_area_name, year):
    """Main function to process protected area boundary analysis"""
    # Initialize classes
    geo_ops = GeometryOperations()
    img_ops = ImageOperations()
    viz = Visualization()

    # Load protected area
    pa_geometry = load_protected_area(protected_area_name).geometry()

    # Process geometry
    aoi = geo_ops.buffer_polygon(pa_geometry)
    aoi = geo_ops.mask_water(aoi)

    # Process imagery
    modis_ic = img_ops.modis.filter(img_ops.filter_for_year(aoi, year))
    band_names = modis_ic.first().bandNames()
    composite = modis_ic.reduce(ee.Reducer.median()).rename(band_names).clip(aoi)
    
    # Add indices and calculate gradient
    image = img_ops.add_indices_to_image(composite)
    single_band = image.select("sur_refl_b03")
    gradient = img_ops.get_gradient_magnitude(single_band)

    # Create select pixels that intersect boundary 
    boundary_pixels = geo_ops.get_pixels_boundary(gradient, pa_geometry, scale=500)
    
    # Visualize
    Map = viz.create_map(pa_geometry, gradient, boundary_pixels)
    
    return Map

In [4]:
Map = main("Serengeti National Park", 2010)
Map

/usr/local/lib/python3.9/site-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for JRC/GSW1_0/GlobalSurfaceWater! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/JRC_GSW1_0_GlobalSurfaceWater

  warnings.warn(warning, category=DeprecationWarning)
/usr/local/lib/python3.9/site-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for MODIS/006/MOD09A1! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD09A1

  warnings.warn(warning, category=DeprecationWarning)


Map(center=[-2.3332666874595325, 34.78445430188864], controls=(WidgetControl(options=['position', 'transparent…